[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/academiannuts/simple_cnn/blob/main/3_deepleaning.ipynb)

In [ ]:
# 必要に応じてinstallする
#!pip install tensorflow
#!pip install tensorflow-gpu
#!pip install keras
#!pip install h5py

# 概要
2_make_dat.ipynbで作成したnumpy配列を用いてCNNで学習済みモデルを作成する

In [ ]:
# config
local_mode = True # local_PC作業時 True, Google_Colab作業時 False
img_size   = 50   # 画像サイズ 
epochs_num = 1    # 学習回数

In [ ]:
# module_import
import tensorflow as tf
import numpy as np
import csv
from keras.utils import np_utils
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import time

In [ ]:
# Diretory_Setting:
if local_mode:
    base_path = './'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/MyDrive/Colab Notebooks/project/'

In [ ]:
# GPU利用可能か確認（for Google_Colab）
tf.test.gpu_device_name()

In [ ]:
# class_list.csvを読み込み
with open(base_path + 'class_list.csv') as f:
    class_list = csv.reader(f)
    classes = []
    for c in class_list:
        classes.append(c[0])
print(classes)     # クラス確認用
num_classes = len(classes)
print(num_classes) # クラス数確認用

In [ ]:
# 訓練データの読み込み
x_train,x_test,y_train,y_test = np.load(base_path+'train_datas.npy', mmap_mode=None, allow_pickle=True , fix_imports=True, encoding='ASCII')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 25
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
# TensorBoard設定
log_dir = base_path + 'log/'
os.makedirs(log_dir, exist_ok=True)  
tensorboard_callback = TensorBoard(log_dir, histogram_freq=1)

# モデル定義
model = Sequential()
model.add(Conv2D(img_size, (3, 3), padding='same',
          input_shape=(img_size, img_size, 3)))    #畳み込み　①
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))                      #畳み込み　②
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))          #プーリング
model.add(Conv2D(64, (3, 3)))                      #畳み込み　③
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))          #プーリング
model.add(Dropout(0.25))                                 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))                                 
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
# 時間計測
startTime = time.time()
# 学習
model.summary()
history = model.fit(x_train, y_train, batch_size=3000, epochs=epochs_num,
                    verbose=1, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])
score = model.evaluate(x_test, y_test, verbose=0)
# 出力
print('loss:', score[0])
print('accuracy:', score[1])
print("Computation time:{0:.3f} sec".format(time.time() - startTime))
#学習済のモデルを保存
model.save(base_path+'trained_model.h5')

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $log_dir